<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> Programación cientifica en Python</h1>
    <h1> Tópico 1: Introducción y numpy</h1>
    <h3> _Abril 2018_</h3>
</center>

_Notebook created by Roberto Fuentes - `roberto.fuentes@alumnos.usm.cl`- DI UTFSM ._

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

from mpl_toolkits.axes_grid1 import make_axes_locatable
import librosa.display
import IPython.display as ipd

## Tabla de contenidos
* [1.- IPython](#ipython)
* [2.- Jupyter Notebook](#jupyter)
* [3.- Magics](#magics)
* [4.- NumPy](#numpy)
* [5.- Dentro de Numpy](#internals) 
* [6.- Programando eficientemente con NumPy](#efficient)

<div id='ipython' />
## 1.- IPython

_IPython_ no es mas que una versión mejorada del shell de _Python_ estándar, que proporciona herramientas para computación interactiva.

Algunas características interesantes de _IPython_ son:

* Mejor resaltado de sintaxis.
* Acceso directo a los comandos bash / linux (`cd`,` ls`, `pwd`,` rm`, `mkdir`, etc.). Los comandos adicionales se pueden ejecutar con: `! Command`.
* comando `who` para ver las variables definidas en la sesión actual.
* Inspeccionar los objetos con `?`.
* Y __magics__, que veremos en breve.

## Algunos ejemplos

In [ ]:
!ls

In [ ]:
!pwd

In [ ]:
!mkdir Ejemplo

In [ ]:
!cd Ejemplo/

In [ ]:
!echo "Soy un ejemplo! :)" > Ejemplo/myfile.txt

In [ ]:
!ls Ejemplo/

In [ ]:
!rm -R Ejemplo/

In [ ]:
who

In [ ]:
?np

<div id='jupyter' />
## 2.- Jupyter Notebook

Es un entorno interactivo basado en web que combina **codigo**, **imagenes**, **videos**, **animaciones**, **matemática**, **lenguaje markdown ** and **gráficos (plot)** en un solo documento. Esta herramienta es un buen comienzo para la computación numérica y el *DataScience* en Python.

# Podemos construir titulos grande

#### o titulos más pequeños

Podemos crear **textos cons links** [links](http://ipython.org)

Tambien podemos escribir ecuaciones como en LaTeX:
$$
E_T = \sum_{l=1}^{N} \sum_{x_k \in \Omega_l} \left( F\left(x_k,S_i(x_k), S_i'(x_k),S_i''(x_k)\right)\right)^2.
$$

Codigo con sintaxis _highlighting_ (resaltado):
```python
def fibonacci(n):
  if n <= 1:
    return n
  else:
    return fibonacci(n-1) + fibonacci(n-2)
```
Podemos mostrar imagenes!: ![This is an image](data/white_fox.jpg)

y distintos plots:

In [ ]:
xgrid = np.linspace(-3,3,50)
f1 = np.exp(-xgrid**2)
f2 = np.tanh(xgrid)
f3 = xgrid/max(xgrid)
fig = plt.figure(figsize=(18, 10))
plt.plot(xgrid, f1, 'bo-', label = "Example curve 1")
plt.plot(xgrid, f2, 'ro-', label = "Example curve 2")
plt.plot(xgrid, f3, 'yo-', label = "Example curve 2")
plt.legend(loc=4, borderaxespad=0., fontsize = 20)
plt.grid()
plt.xlabel(r"$x$",fontsize = 20)
plt.ylabel(r"$y$",fontsize = 20)
plt.title(r"Just a demo plot for $e^x$, $\tanh(x)$ and normalized $y = x$ function",fontsize = 20)
plt.show()

_Ipython_ tambien cuenta con un sofisticado sistema de *display* que nos permite insertar elementos web en los _notebooks_. Aqui tenemos por ejemplo un video de **Youtube** mostrado directamente en el _notebook_:  

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('uwmeH6Rnj2E')

Incluso podemos mostrar audio en distintos formatos:

In [ ]:
x, sr = librosa.load('data/dejavu.wav')

In [ ]:
ipd.Audio(x, rate=sr)

<div id='magics' />
## 3.- Magics

Los _magics_ son comandos "customizados" que permiten interactuar directamente con nuestro SO y el sistema de archivos del computador. Existen dos tipos de *line magics*:

* `%` (El cual solo afecta a la linea en custion).
* `%%` (El cual afecta a toda la celda).

Aqui tenemos algunos `magics` útiles:

In [ ]:
# this will list all magic commands
%lsmagic

In [ ]:
# also work in ls, cd, mkdir, etc
%pwd

In [ ]:
%history

In [ ]:
# this will execute and show the output of the program
%run ./data/hola_mundo.py

In [ ]:
def Evitame():
    for i in range(100):
        for j in range(100):
            for k in range(100):
                a = 1+1
    return None

%timeit -n 10 Evitame()

In [ ]:
%time Evitame()

In [ ]:
%%bash
cd ..
ls

In [ ]:
%%writefile myfile.txt
R: Holanda que talca como andamio que container que acelga!
P: No sapa naipe hermanito.

In [ ]:
!cat myfile.txt
!rm myfile.txt

### Escribe tus propios _magics_ !

Ahora veremos como crear un nuevo _cell magic_ que compile y ejecute codigo C++ en nuestro notebook: 

In [ ]:
from IPython.core.magic import register_cell_magic

Para comenzar, crearemos una funcion que tome una linea y el contenido de una celda como sus argumentos, y la *decoraremos* con `@register_cell_magic`.

In [ ]:
@register_cell_magic
def cpp(line,cell):
    """Compile, execute C++ code, and return the
    standard output."""
    # We first retrieve the current IPython interpreter instance.
    ip = get_ipython()
    # We define the source and executable filenames.
    source_filename = '_temp.cpp'
    program_filename = '_temp'
    # We write the code to the C++ file.
    with open(source_filename, 'w') as f:
        f.write(cell)
    # We compile the C++ code into an executable.
    compile = ip.getoutput("g++ {0:s} -o {1:s}".format(
              source_filename, program_filename))
    # We execute the executable and return the output.
    output = ip.getoutput('./{0:s}'.format(program_filename))
    print(output[0])

In [ ]:
%%cpp
#include<iostream>
int main() 
{
    std::cout << "Hello world!";
}

Sin embargo esta _cell magic_ es valida solo dentro del _notebook_ donde se creó. Para usarla en otros _notebooks_ se puede crear una extensión de _Ipython_ (**IPython extension**). Para esto, copie la definición de la función `cpp()` (sin el decorador) en un modulo de Python, llamada `cpp_ext.py` para este ejemplo. Luego, agregue esto al final de su archivo:

```python
def load_ipython_extension(ipython):
       """This function is called when the extension is loaded.
       It accepts an IPython InteractiveShell instance.
       We can register the magic with the `register_magic_function`
       method of the shell instance."""
       ipython.register_magic_function(cpp, 'cell')
```

Puedes cargar la extensión con %load_ext cpp_ext. Recuerda que tu archivo cpp_ext.py debe estar donde se encuentre tu _notebook_ (mismo directorio).

In [ ]:
%load_ext cpp_ext

In [ ]:
%%cpp
#include<iostream>
int main() 
{
    std::cout << "Hello world!";
}

## Actividad 1

Desarrolle una extensión de `IPython` (llamada `2to3_ext.py`) implementando un `cell magic`, que reciba en su celda un código en `Python2`, lo transforme a código compatible con `Python3`, y finalmente lo ejecute en una instancia del `IPython3`, imprimiendo en la salida. 

La entrega consiste en la extensión (`2to3_ext.py`) + un notebook de ejemplo de su extensión ejecutada con el código de a continuación:

```python
a = 3
b = 4
c = 5

if a>(b+c) or b>(a+c) or c>(a+b):
	print "Ingrese un traingulo valido."
elif a==b and b==c:
	print "Triangulo equilatero."
elif a==b or b==c or a==c:
	print "Triangulo isoceles."
else:
	print "Triangulo escaleno."
```

**Nota:** Consideraremos que las únicas diferencias entre `Python2` y `Python3` son los `print`.

<div id='numpy' />
# 4.- NumPy

## Operadores básicos en _NumPy_

La razon de porque se debe usar _NumPy_ en vez de un objeto iterable en Python son:
* _NumPy_ provee una estructura de arreglos (_ndarrays_) que guarda la data **de forma contigua**.
* _NumPy_ tiene implemnetado **operaciones matemáticas muy veloces** sobre los _ndarrays_ que explota el formato de guardado de estos (forma contigua).
* **Sintaxis corto para operaciones entre arrays**. Un lenguaje como C o Java requiere de un for para escribir operaciones tan simples como $C = A + B$.

### Creando nuestros primeros arreglos

Hay muchas funciones en _NumPy_ para crear distintos tipos de arreglos. A continuación se muestra una lista de los comandos más usados:

In [ ]:
# Arrays of zeros: np.zeros(shape)
print("Matriz de ceros:")
print( np.zeros((3,3)) )

# Arrays of ones: np.ones(shape)
print("\nMatriz de unos:")
print( np.ones((3,3)) )

# Empty array: np.empty(shape)
print("\nArreglos vacios:")
print( np.empty((3,3)) )

# Range of values: np.range(start, stop, step)
print("\nRange:")
print( np.arange(0., 10., 1.) )

# Regular grid: np.linspace(start, end, n_values)
print("\nGrilla Regular:")
print( np.linspace(0., 1., 9) )

# Random secuences: np.random
print("\nSecuencia Random:")
print( np.random.uniform(10, size=6) )

# Array constructor: np.array( python_iterable )
print("\nConstrucción de un array en NumPy")
print( np.array([2, 3, 5, 10, -1]) )

### Operaciones básicas

Muchas de las operaciones en _NumPy_ son del tipo **element-wise**, es decir, realizar el computo $C = A + B$ se traduce en $C[i,j] = A[i,j] + B[i,j]$. A continuación se muestran algunas de las operaciones mas usuales (Para ver más funciones puede entrar aqui: [NumPy mathematical functions](https://docs.scipy.org/doc/numpy/reference/routines.math.html). )

In [ ]:
# first we create two random arrays:
A = np.random.random((5,5))
B = np.random.random((5,5))

# sum
print("Suma:")
print( A+B )

# subtraction
print("\nResta")
print( A-B )

# product
print("\nProducto")
print( A*B )

# matricial product
print("\nProducto Matricial ")
print( np.dot(A,B) )

# power
print("\nPotencia")
print( A**2 )

# Some common mathematical functions
print("\n np.exp()")
print( np.exp(A) )
print("\n np.sin()")
print( np.sin(A) )
print("\n np.cos()")
print( np.cos(A))
print("\n np.tan()")
print( np.tan(A) )

### Operaciones booleanas

Las comparaciones en NumPy trabajan tambien de forma _element wise_. Algunos ejemplos son:

In [ ]:
# Creating two 2d-arrays
A = np.array( [[1, 2, 3], [2, 3, 5], [1, 9, 6]] )
B = np.array( [[1, 2, 3], [3, 5, 5], [0, 8, 5]] )

print("A > B:")
print( A > B )

print("\nA =< B:")
print( A <= B )

print("\n A==B:")
print( A==B )

print("\n A!=B:")
print( A!=B )

# Creating two 2d boolean arrays
C = A==B
D = A>=B

print("\n A and B:")
print( C & D)
print( np.logical_and(C,D) )

print("\n A or B:")
print( C | D)
print( np.logical_or(C,D) )

print("\n not A:")
print( ~C )
print( np.logical_not(C))

<div id='internals' />
## Dentro de NumPy

### La estructura de los  `numpy.ndarray` 

_ndarray_ es un objeto de NumPy que nos permite crear un arreglo de $N$ dimensiones. Basicamente esta formado por:

1. Un número de **dimensiones**.
2. El porte de cada dimension (_shape_).
3. El **tipo de dato** (_dtype_).
4. El buffer de la data.
<img src='data/ndarray.png' style="width: 500px;">

### Veamos un problema interesante ...
Consideremos calcular la norma cuadradada de un vector ($\displaystyle || \mathbf{v} ||^2 = \mathbf{v} \cdot \mathbf{v}$) con las siguientes 4 implementaciones:

In [ ]:
#Python Lists implementation
def norm_square_list(vector):
    norm = 0
    for v in vector:
        norm += v*v
    return norm

#Naive NumPy implementation
def norm_square_array(vector):
    norm = 0
    for v in vector:
        norm += v*v
    return norm

#Vectorized NumPy implementation
def norm_square_numpy(vector):
    return np.sum(vector * vector)

#Clever NumPy implementation
def norm_square_dot(vector):
    return np.dot(vector, vector)

#Vector to use - dimension 10^6
vector = range(1000000)
npvector = np.array(vector)

In [ ]:
#Timing the list implementation
%timeit norm_square_list(vector)

In [ ]:
#Timing the naive array implementation
%timeit norm_square_array(npvector)

In [ ]:
#Timing the NumPy-vectorized implementation
%timeit norm_square_numpy(npvector)

In [ ]:
#Timing the clever NumPy-vectorized implementation
%timeit norm_square_dot(npvector)

Como se puede ver, suceden cosas interesantes. Vamos por partes "como dijo el descuartizador": 
* La implementación inicial en NumPy (*norm_square_array*) la cual itera sobre la data es actualmente **peor** que simplemente usar una lista en python. Esto ocurre porque el array es guardado en una representación de bajo nivel, y **debe ser convertido** en un objeto compatible con Python antes de ser retornado al usuario, causando una sobrecarga extra cada vez que accedes a un indice del arreglo.

* `norm_square_numpy` es mas lenta que la implementación _cleaver_ por dos razones:
    1. Hay un delta de tiempo que se gasta en asignar memoria para guardar el resultado temporal (vector x vector).
    2. Lo anterior crea dos loops implicitos, uno para la multiplicación y otro para la suma.
* La implementación _cleaver_ usa la función de NumPy *np.dot()*, la cual no necesita guardar los resultados intermedios, e itera una sola vez (a la velocidad del lenguaje C).

<div id='efficient' />
## Programando eficientemente con NumPy

### Operaciones de copiado _In-place_ e implicitas

Siempre que puedas, escoge las operaciones **in-place** sobre las **copias implicitas**. Esto te ayudara a ahorrar memoria (lo cual es menos trabajo para el pobre recolector de basura =)! ) y ejecutara de forma mas rápida tu código.

In [ ]:
def id(x):
    """
    This function returns the memory
    block address of an array.
    """
    return x.__array_interface__['data'][0]

La computación de arreglos puede involucrar operaciones **in-place** (primer ejemplo: el arreglo se modifica) o las **copias implicitas** (segundo ejemplo: un nuevo arreglo es creado).

In [ ]:
a = np.zeros(10); aid = id(a)

In [ ]:
# in-place operation
a *= 2; id(a) == aid

In [ ]:
# implicit-copy operation
a = a * 2; id(a) == aid

In [ ]:
%%timeit 
a = np.ones(100000000)
a *= 2

In [ ]:
%%timeit
a = np.ones(100000000)
b = a * 2

## Broadcasting

No es necesario realizar un *reshape* sobre los arreglos para operar sobre ellos. *Broadcasting* es una característica de los arreglos de NumPy. En la imagen que se muestra acontinuación la memoria extra indicada por los cuadros con puntos nunca se asigna, pero puede ser conveniente pensar en las operaciones como si fuera así.
<img src='data/broadcasting.png' style="width: 600px;">

__Como funciona:__ Los dos arreglos que estan involucrados en esta operación deben tener en común al menos una dimensión. El arreglo con menor dimensión se extenderá de forma **lógica** para hacer el *match* con la dimensión del otro arreglo.

In [ ]:
# array([0,1,2]) + 5
np.arange(3) + 5

In [ ]:
# array([[1, 1 ,1], [1, 1, 1], [1, 1, 1]]) + array([0, 1, 2])
np.ones((3,3)) + np.arange(3)

In [ ]:
# array([[0], [1], [2]]) + array([0, 1 ,2])
np.arange(3).reshape((3,1)) + np.arange(3)

## Máscaras booleanas

Una técnica para acceder a un arreglo sin loops son las máscaras booleanas. Supongamos que queremos obtener todos los elementos de un arreglo menores que 0.5: 

In [ ]:
def naive_indexing(vect):
    ret = list()
    for val in vect:
        if val < 0.5: ret.append(val)
    return np.array(ret)

#data to occupy and mask of booleans
vect = np.random.random_sample(1000000)
mask = vect < 0.5
mask

In [ ]:
#naive indexing
%timeit naive_indexing(vect)

In [ ]:
#mask indexing
%timeit vect[mask]

In [ ]:
naive_indexing(vect)

In [ ]:
vect[mask]

## Actividad #2

In [ ]:
def image_plot(data, title='FITS image'):
    plt.figure(figsize=(20,20))
    im = plt.imshow(data, cmap=plt.cm.afmhot, interpolation=None)
    plt.title(title)
    #plt.axis('off')
    divider = make_axes_locatable(plt.gca())
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
def comparation_plot(img1, img2, etiqueta_1, etiqueta_2):
    fig = plt.figure(figsize=(20,20))
    a = fig.add_subplot(1,2,1)
    plt.imshow(img1, cmap='gray', interpolation=None)
    a.set_title(etiqueta_1)
    
    b = fig.add_subplot(1,2,2)
    plt.imshow(img2, cmap='gray', interpolation=None)
    b.set_title(etiqueta_2)

    plt.show()

In [ ]:
# first we load the data:
data = np.load("data/red_fox.npy")

In [ ]:
image_plot(data)

Usted deberá aplicar un filtro a la presente imagen. Para esto, debe separar su imagen en los 3 canales respectivos (Red, Green, Blue - RGB), trabajar con cada canal por separado y luego juntarlos. Los pasos que debe seguir por cada canal son los siguientes:

### Paso 1) 

Calcular el `RMS` de la imagen entregada. 

$$RMS = \sqrt{\frac{1}{m\ n} \sum_{i=1}^m \sum_{j=1}^n \texttt{data[i,j]}^2}$$

__Nota:__ Computarlo de forma vectorizada.

### Paso 2) 

Genere otro arreglo donde los pixeles con intensidades por debajo del `RMS` son considerados como _no usables_ (con valor `=0`).  Mostrar tal imagen resultante.

### Paso 3)

Crear la función
```python
def apply_filter(data, mask, kernel_filter):
    ...
    return None
```
Que reciba el arreglo de datos completo `data`, el arreglo booleano con los __pixeles usables__ `mask` (sobre el RMS), y kernel de filtro de `3x3`. La función debe convolucionar `filter` sobre la imagen `data`, sólo en los pixeles usables. La función debe retornar la data modificada.

Finalmente mostrar el resultado de convolucionar tal filtro en `data` (mostrar imágen).

__Nota:__ Debe usar siempre que pueda _instrucciones vectorizadas_.

__Image convolution:__ https://en.wikipedia.org/wiki/Kernel_(image_processing)#Convolution

In [ ]:
# Gaussian blur filter 3x3: Ocupar este filtro!
kernel_filter = 1./16. * np.array([[1,2,1], [2,4,2], [1,2,1]])
print(kernel_filter)